In [2]:
from sequence_example_lib import *
import os
import tensorflow as tf
import numpy as np
import time

sequence_example_file = ('/Users/mac/Desktop/Brain/Simplified_Magenta/Wikifonia_basic_rnn_sequence_examples/training_melodies.tfrecord')
sequence_example_file_paths = tf.gfile.Glob(
    os.path.expanduser(sequence_example_file))

start_time = time.time()
inputs, labels, lengths = get_numpy_from_tf_sequence_example( input_size=38,
                                    sequence_example_file_paths = sequence_example_file_paths,
                                    shuffle = False)
print('Time:',time.time() - start_time)
print('inputs shape',inputs.shape)
print('inputs type',type(inputs))
print('Finished! And then use ipdb for debug and watch the variables')

INFO:tensorflow:Counting records in /Users/mac/Desktop/Brain/Simplified_Magenta/Wikifonia_basic_rnn_sequence_examples/training_melodies.tfrecord.
INFO:tensorflow:Total records: 6668
INFO:tensorflow:[<tf.Tensor 'ParseSingleSequenceExample_1/ParseSingleSequenceExample:0' shape=(?, 38) dtype=float32>, <tf.Tensor 'ParseSingleSequenceExample_1/ParseSingleSequenceExample:1' shape=(?,) dtype=int64>, <tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>]
inputs (6668, 511, 38) <class 'numpy.ndarray'> [ 0  0  0  0  0  0  0  0 18  0]
labels (6668, 511) <class 'numpy.ndarray'> [ 0  0  0  0  0  0  0 18  0  0]
Time: 2.771904945373535
inputs shape (6668, 511, 38)
inputs type <class 'numpy.ndarray'>
Finished! And then use ipdb for debug and watch the variables


In [ ]:
def to_events(x):
    return np.argmax(x, axis=1)

In [ ]:
input_events = []
for i in inputs:
    input_events.append(to_events(i))

In [ ]:
def to_real_length(x):
    while(x[len(x)-1]==0):
        x.pop()

In [ ]:
real_inputs = []
for i in input_events:
    d = list(i)
    to_real_length(d)
    real_inputs.append(d)

In [ ]:
real_length = []
for i in real_inputs:
    real_length.append(len(i))
real_length = np.array(real_length)

In [ ]:
print(real_length.min())
print(real_length.max())
print(real_length.mean())

In [ ]:
file = open("len.txt","w")
for i in real_length:
    file.write("%d\n"%i)
file.close()

In [ ]:
def zero_rate(x):
    x = np.array(x)
    non_zero_num = len(list(x.nonzero()[0]))
    return 1-non_zero_num*1.0/len(x)

In [ ]:
zero_rates = []
for i in real_inputs:
    zero_rates.append(zero_rate(i))
zero_rates = np.array(zero_rates)

In [ ]:
print(zero_rates.min())
print(zero_rates.max())
print(zero_rates.mean())

In [ ]:
for i in range(3):
    print(real_inputs[i])

In [ ]:
from my_to_midi import *

In [ ]:
for i in range(1,11):
    a = real_inputs[10]
    events_to_midi(a, 'test_midi', 'input%d'%i)

In [ ]:
for i in range(len(real_inputs)):
    real_inputs[i].pop()

In [ ]:
import pickle
real_inputs_file = open("real_inputs.pkl","wb")
pickle.dump(real_inputs, real_inputs_file)
real_inputs_file.close()

In [ ]:
test_file = open("real_inputs.pkl","rb")
a = []
a = pickle.load(test_file)
test_file.close()

In [ ]:
index_to_char = "._0123456789abcdefghijklmnopqrstuvwxyz"
with open("Wikifonia_to_text.txt", "w") as wf:
    for i in real_inputs:
        for j in range(len(i)):
            wf.write(index_to_char[i[j]])
    wf.close()